# Recurrent Neural Network example

[Original video](https://youtu.be/Gl2WXLIMvKA)

Input of RNN is Nx28x28 --> 28 time sequences with 28 features each.

N x time_sequence x features

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [ ]:
# Set deterministic
seed = 0
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters
input_size = 28  # number of features for each time step
sequence_length = 28
num_layers = 2
hidden_size = 256  # number of nodes in each time step
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

## RNN

In [ ]:
# Create a RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        # init hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagation
        out, _ = self.rnn(x, h0)  # "_" => ignore the hidden state output
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
# Check if it runs correctly
# MNIST 28x28=784, output classes = 10
model = RNN(input_size=input_size, hidden_size=hidden_size,
            num_layers=num_layers, num_classes=num_classes).to(device)

x = torch.randn(batch_size, 28, 28).to(device)

# Run model on the input and print the shape
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Initialize network
model = RNN(input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, num_classes=num_classes).to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent or adam step
        optimizer.step()

In [ ]:
# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()  # set model into train mode
    #return acc

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58295 / 60000 with accuracy 97.16
Checking accuracy on test data
Got 9717 / 10000 with accuracy 97.17


In [ ]:
# Without gradient clipping (a little bit lower accuracy)
#   Checking accuracy on training data
#   Got 57072 / 60000 with accuracy 95.12
#   Checking accuracy on test data
#   Got 9517 / 10000 with accuracy 95.17

# Gradient clipping ( a little bit better)
#   Checking accuracy on training data
#   Got 58295 / 60000 with accuracy 97.16
#   Checking accuracy on test data
#   Got 9717 / 10000 with accuracy 97.17

## GRU

In [ ]:
# Create a GRU
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        # init hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagation
        out, _ = self.gru(x, h0)  # "_" => ignore the hidden state output
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
# Initialize network
model = GRU(input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, num_classes=num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent or adam step
        optimizer.step()

# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()  # set model into train mode
    #return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59210 / 60000 with accuracy 98.68
Checking accuracy on test data
Got 9857 / 10000 with accuracy 98.57


## LSTM

In [ ]:
# Create a LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        # init hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # init serapate cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagation
        out, _ = self.lstm(x, (h0, c0))  # "_" => ignore the hidden state output
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

# Initialize network
model = LSTM(input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, num_classes=num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent or adam step
        optimizer.step()

# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()  # set model into train mode
    #return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59182 / 60000 with accuracy 98.64
Checking accuracy on test data
Got 9841 / 10000 with accuracy 98.41


## LSTM with only the last hidden state

In [ ]:
# Use information only from the last hidden state and ignore other hidden states.

# Create a LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        #self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # init hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # init serapate cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagation
        out, _ = self.lstm(x, (h0, c0))  # "_" => ignore the hidden state output
        #out = out.reshape(out.shape[0], -1)
        #print(out.shape)  # torch.Size([64, 28, 256])
        #print(out[:,-1,:].shape)  # torch.Size([64, 256])
        out = self.fc(out[:,-1,:])
        return out

# Initialize network
model = LSTM(input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, num_classes=num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent or adam step
        optimizer.step()

# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()  # set model into train mode
    #return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58950 / 60000 with accuracy 98.25
Checking accuracy on test data
Got 9785 / 10000 with accuracy 97.85
